In [3]:
import pandas as pd
import re


In [16]:
# load the CSV file into a DataFrame
df = pd.read_csv('data/culturing_2024.csv')

# Define a regex pattern to match '*number* RQ' and common misspellings like '12 RQ,'
rq_pattern = re.compile(r'(\d+)\s*RQ\W*', re.IGNORECASE)

# Apply the pattern to the 'ItemComment' column to find all matches
df['RQ_Matches'] = df['ItemComment'].str.findall(rq_pattern)

# Ensure that NaN values in 'RQ_Matches' are treated as empty lists
df['RQ_Matches'] = df['RQ_Matches'].apply(lambda x: x if isinstance(x, list) else [])

# Remove the rows without 'RQ' pattern from the dataframe
df_cleaned = df[~df['RQ_Matches'].apply(lambda x: len(x) == 0)]

# When creating df_cleaned from another DataFrame (df in this case), use .copy()
df_cleaned = df[~df['RQ_Matches'].apply(lambda x: len(x) == 0)].copy()


# Extract the numbers from the 'RQ' pattern, cast to integers, and sum
total_rq = df_cleaned['RQ_Matches'].explode().astype(int).sum()

total_rq


3866

In [17]:
# Count the number of missing 'RQ' entries
missing_rq_count = missing_rq.sum()

# Get the total number of rows for reference
total_rows = df.shape[0]

# Filter the dataframe for rows where 'RQ' pattern is missing
rows_without_rq = df[missing_rq]

# Extract the 'ItemAccNoFull' for those rows
item_acc_no_full_without_rq = rows_without_rq['ItemAccNoFull'].tolist()


In [18]:
item_acc_no_full_without_rq

['2022-0007.87',
 '2022-0163.87',
 '2022-0332.88',
 '2021-0820.87',
 '2023-0286.88',
 '2023-0306.88',
 '2023-0389.88']

In [19]:
# To group the number of plants requested by 'LifeForm', we'll first need to aggregate the RQ numbers per 'LifeForm'

# Extract the first match of RQ numbers (assuming there's only one RQ per comment)
# and convert it to integer
df_cleaned.loc[:, 'RQ_Number'] = df_cleaned['RQ_Matches'].apply(lambda x: int(x[0]) if x else 0)

# Now, group by 'LifeForm' and sum the 'RQ_Number' for each group
rq_per_lifeform = df_cleaned.groupby('LifeForm')['RQ_Number'].sum().reset_index()

rq_per_lifeform


,LifeForm,RQ_Number
0,Annual,32
1,"Bulb, Corm, or Tuber",50
2,"Climber, Liana, or Vine",28
3,Herbaceous Perennial,1656
4,Shrub,383
5,Shrub or Tree,181
6,Tree,376


In [20]:
# Group the number of plants requested by 'Family', summing up the 'RQ_Number' for each family
rq_per_family = df_cleaned.groupby('Family')['RQ_Number'].sum().reset_index()

rq_per_family


,Family,RQ_Number
0,Amaryllidaceae,9
1,Anacardiaceae,6
2,Apiaceae,87
3,Apocynaceae,6
4,Aquifoliaceae,14
...,...,...
74,Theaceae,12
75,Tofieldiaceae,12
76,Ulmaceae,4
77,Verbenaceae,50


In [21]:
# Group the number of plants requested by 'Purpose', taking into account possible empty values
# First, fill empty values in the 'Purpose' column with a placeholder like "Unknown"
df_cleaned['Purpose'].fillna('Unknown', inplace=True)

# Now, group by 'Purpose' and sum the 'RQ_Number' for each purpose
rq_per_purpose = df_cleaned.groupby('Purpose')['RQ_Number'].sum().reset_index()

rq_per_purpose


,Purpose,RQ_Number
0,1B,15
1,1C,440
2,1F,235
3,1J,663
4,"1J, LNA",188
5,1P,54
6,1P/3A,3
7,2A,541
8,3A,784
9,4C,237


In [22]:
# Filter the DataFrame for entries with 'Unknown' purpose
unknown_purpose_entries = df_cleaned[df_cleaned['Purpose'] == 'Unknown']

# Extract the 'ItemAccNoFull' for these entries
item_acc_no_full_unknown_purpose = unknown_purpose_entries['ItemAccNoFull'].tolist()

item_acc_no_full_unknown_purpose


['2022-0493.88',
 '2022-0520.88',
 '2021-0643.88',
 '2021-0644.88',
 '2022-0483.88',
 '2022-0491.88',
 '2022-0496.88',
 '2022-0500.88',
 '2022-0513.88',
 '2022-0517.88',
 '2022-0524.88',
 '2022-0537.88',
 '2022-0542.85',
 '2022-0484.87',
 '2022-0479.88',
 '2022-0534.88',
 '2022-0486.88',
 '2022-0542.88',
 '2022-0509.87',
 '2022-0509.88',
 '2022-0532.88',
 '2023-0271.88',
 '2022-0534.86',
 '2022-0486.87',
 '2022-0484.86',
 '2022-0542.86',
 '2022-0507.87',
 '2023-0386.88']

Provenance

In [23]:
# Group the number of plants requested by 'ProvenanceCode', taking into account possible empty values
# First, fill empty values in the 'ProvenanceCode' column with a placeholder like "Unknown"
df_cleaned['ProvenanceCode'].fillna('Unknown', inplace=True)

# Now, group by 'ProvenanceCode' and sum the 'RQ_Number' for each provenance code
rq_per_provenance = df_cleaned.groupby('ProvenanceCode')['RQ_Number'].sum().reset_index()

rq_per_provenance


,ProvenanceCode,RQ_Number
0,G,1659
1,U,301
2,W,1785
3,Z,121


Purpose

In [24]:
# Correcting the entry for "1P/3A" before proceeding with the groupings
df_cleaned.loc[df_cleaned['Purpose'] == '1P/3A', 'Purpose'] = '1P, 3A'

# Extracting the 'ItemAccNoFull' for the corrected entry
item_acc_no_full_corrected = df_cleaned[df_cleaned['Purpose'] == '1P, 3A']['ItemAccNoFull'].tolist()

# Mapping purposes to umbrella groups
purpose_to_group = {
    'L': 'Alpine Garden',
    '1J': 'North America',
    '1B': 'North America',
    '1G': 'North America',
    'NAM': 'North America',
    '4C': 'North America',
    '3A': 'Asian Garden',
    '1F': 'Physic Garden',
    'F': 'Physic Garden',
    '1P': 'Winter Garden',
    '1C': 'Contemporary Garden',
    '2A': 'Front Entrance'
}

# Function to assign umbrella group to each purpose, considering multiple entries
def assign_umbrella_group(purpose_str):
    purposes = purpose_str.split(', ')
    groups = set()
    for purpose in purposes:
        # Check for direct match or startswith for entries like "LEU" (which should match "L")
        group = next((group_name for code, group_name in purpose_to_group.items() if purpose == code or purpose.startswith(code)), None)
        if group:
            groups.add(group)
    return list(groups)

# Apply the function to assign umbrella groups
df_cleaned['UmbrellaGroup'] = df_cleaned['Purpose'].apply(assign_umbrella_group)

# For each row, distribute the RQ_Number evenly among its umbrella groups
from collections import defaultdict
group_totals = defaultdict(int)

for index, row in df_cleaned.iterrows():
    num_groups = len(row['UmbrellaGroup'])
    split_rq = int(row['RQ_Number'] / num_groups) if num_groups > 0 else 0
    for group in row['UmbrellaGroup']:
        group_totals[group] += split_rq

group_totals, item_acc_no_full_corrected


(defaultdict(int,
             {'North America': 1021,
              'Asian Garden': 785,
              'Alpine Garden': 510,
              'Winter Garden': 55,
              'Physic Garden': 259,
              'Contemporary Garden': 440,
              'Front Entrance': 541}),
 ['2022-0331.88'])

In [25]:

# Convert the dictionary to a DataFrame
group_totals_df = pd.DataFrame(list(group_totals.items()), columns=['Garden Area', 'Total Requests'])

group_totals_df


,Garden Area,Total Requests
0,North America,1021
1,Asian Garden,785
2,Alpine Garden,510
3,Winter Garden,55
4,Physic Garden,259
5,Contemporary Garden,440
6,Front Entrance,541


Unique Values


In [26]:
# Count the number of unique values in the "PropTreatment" column
unique_prop_treatment = df_cleaned['PropTreatment'].nunique()

unique_prop_treatment


35

In [27]:
# List the unique values in the "PropTreatment" column alphabetically
unique_prop_treatment_values = sorted(df_cleaned['PropTreatment'].dropna().unique())

unique_prop_treatment_values


['Cold stratification',
 'Cold stratification (x day)',
 'Cold stratification (x day), cool to germinate',
 'Cold stratification (x day).',
 'Cool germinator. 5C night/ 10C day',
 'Double dormancy. 2-year cycle. sow asap.',
 'Fire, 24hr water soak.',
 'Surface sown on milled sphagnum. Cold stratification. Cool to germinate.',
 'Surface sown on sphagnum with glass plate on top of pot. Cold Stratification.',
 'Surface sown on sphagnum with glass plate on top of pot. mist lightly once a week until prothalli appear then increase mist duration.',
 'Warm stratification (x days), cold stratification (x days)',
 'Warm stratification (x days), cold stratification (x days), 2 year germination (double dormancy)',
 'light cover, 10C',
 'light cover, 20C day / 10C night',
 'light cover, 20C.',
 'light cover, 20C. if no germination in 30 days then 30 day CS',
 'light cover, cold stratification (x) days',
 'light cover, cool to germinate.',
 'mix w/t moist perlite, store in refrigerator, check weekly

In [31]:
# Define a function that takes a "PropTreatment" string and returns all 'AccNoFull' that contain it
def get_acc_no_full_by_treatment(treatment):
    # Filter the DataFrame for entries that contain the specified treatment in "PropTreatment"
    matching_entries = df_cleaned[df_cleaned['PropTreatment'].str.contains(treatment, na=False)]
    
    # Extract the 'AccNoFull' for these entries
    acc_no_full = matching_entries['AccNoFull'].tolist()
    
    return acc_no_full

# Example usage of the function
result = get_acc_no_full_by_treatment('light cover, 20C. if no germination in 30 days then 30 day CS')
print(result)

# Note: The example usage is commented out to avoid running it directly here. You can use this function by
# uncommenting the example usage and replacing the "Cold stratification" with any treatment string you're interested in.


['2022-0572', '2024-0021']


In [33]:
unique_prop_treatment_values = sorted(df_cleaned['PropComment'].dropna().unique())

unique_prop_treatment_values

['?PD',
 '?PD or ND',
 'MPD',
 'MPD, cool to G',
 'MPD, dark+',
 'MPD, heat',
 'MPD?',
 'ND',
 'ND or MPD, 2-yr',
 'ND or PD',
 'ND or light PD',
 'ND, G. <10C',
 'ND, G.<10C',
 'ND, G<5C, +2 yr',
 'ND, Light+',
 'ND, cool',
 'ND, cool to G.',
 'ND, cool to G., erratic',
 'ND, light',
 'ND, light+',
 'ND, light, cool to G.',
 'ND, smoke',
 'ND, warm',
 'ND. light',
 'PD',
 'PD or ND',
 'PD or ND, seed coat',
 'PD, aquatic',
 'PD, cool',
 'PD, cool to G.',
 'PD, dark+',
 'PD, do not dewing',
 'PD, light',
 'PD, light+',
 'PD, light, cool to G.',
 'PD, recalcitrant',
 'PD, smoke',
 'PD?',
 'PY',
 'PY + PD',
 'PY, MPD',
 'PY, MPD, smoke',
 'PY, ND or light PD?',
 'PY, PD',
 'PY, PD, winter',
 'PY, fire',
 'Py, PD',
 'no info, typica G.l > 25C']

In [38]:
# Define a function that takes a "PropTreatment" string and returns all 'AccNoFull' that contain it
def get_acc_no_full_by_treatment(treatment):
    # Filter the DataFrame for entries that contain the specified treatment in "PropTreatment"
    matching_entries = df_cleaned[df_cleaned['PropComment'].str.contains(treatment, na=False)]
    
    # Extract the 'AccNoFull' for these entries
    acc_no_full = matching_entries['AccNoFull'].tolist()
    
    return acc_no_full

# Example usage of the function
result = get_acc_no_full_by_treatment('no info, typica G.l > 25C')
print(result)

# Note: The example usage is commented out to avoid running it directly here. You can use this function by
# uncommenting the example usage and replacing the "Cold stratification" with any treatment string you're interested in.

['2022-0046']


In [32]:
from sklearn.preprocessing import MultiLabelBinarizer

# Split the 'PropComment' entries by comma and strip extra spaces
df_cleaned['SplitPropComment'] = df_cleaned['PropComment'].str.split(',').apply(lambda x: [i.strip() for i in x] if isinstance(x, list) else [])

# Initialize MultiLabelBinarizer to handle the creation of dummy variables
mlb = MultiLabelBinarizer()

# Apply MultiLabelBinarizer to the list of lists (each list representing the conditions mentioned in 'PropComment' for a row)
mlb_result = mlb.fit_transform(df_cleaned['SplitPropComment'])

# Create a DataFrame from the result
prop_comment_encoded_df = pd.DataFrame(mlb_result, columns=mlb.classes_)

# Concatenate the original DataFrame with the new one-hot encoded DataFrame
df_cleaned_encoded = pd.concat([df_cleaned.reset_index(drop=True), prop_comment_encoded_df], axis=1)

# Showing the new columns added to the DataFrame as a result of one-hot encoding
encoded_columns = prop_comment_encoded_df.columns.tolist()
encoded_columns


['+2 yr',
 '2-yr',
 '?PD',
 '?PD or ND',
 'G. <10C',
 'G.<10C',
 'G<5C',
 'Light+',
 'MPD',
 'MPD?',
 'ND',
 'ND or MPD',
 'ND or PD',
 'ND or light PD',
 'ND or light PD?',
 'ND. light',
 'PD',
 'PD or ND',
 'PD?',
 'PY',
 'PY + PD',
 'Py',
 'aquatic',
 'cool',
 'cool to G',
 'cool to G.',
 'dark+',
 'do not dewing',
 'erratic',
 'fire',
 'heat',
 'light',
 'light+',
 'no info',
 'recalcitrant',
 'seed coat',
 'smoke',
 'typica G.l > 25C',
 'warm',
 'winter']